In [107]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split #для разделения на тренировочные и тестовые выборки
from tqdm import tqdm
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
def make_regression_data(n=100, noise=0.2, source='random'):
    if source== 'random':
        X = torch.randn(n,1)
        w,b = -5, 10
        y = w*X + b + noise * torch.randn(n, 1)
        return X,y
    
    elif source == 'diabetes':
        from sklearn.datasets import load_diabetes
        data = load_diabetes()
        X = torch.tensor(data['data'], dtype=torch.float32)
        y = torch.tensor(data['target'], dtype=torch.float32).unsqueeze(1)
        return X, y
    else:
        raise ValueError('Unk sources')
        
def mse(y_pred: torch.Tensor, y_true:torch.Tensor) -> torch.Tensor:
    return ((y_pred - y_true)**2).mean()
        
def log_epoch(epoch, avg_loss, **metrics):
    message = f'Epoch: {epoch}\t loss:{avg_loss}'
    for k, v in metrics.items():
        message += f'\t{k}: {v:.4f}'
    print(message)
    
def make_classification_data(n = 100):
    from sklearn.datasets import load_breast_cancer
    data = load_breast_cancer()
    X = torch.tensor(data['data'], dtype=torch.float32)
    y = torch.tensor(data['target'], dtype=torch.float32).unsqueeze(1)
    return X, y

def make_multiclass_data(): #датасет с множеством классов
    from sklearn.datasets import load_wine
    data = load_wine()
    X = torch.tensor(data.data, dtype=torch.float32)
    y = torch.tensor(data.target, dtype=torch.long) 
    return X, y

def accuracy(y_pred, y_true):
    y_pred_bin = (y_pred>0.5).float()
    return (y_pred_bin == y_true).float().mean().item()

# №1.1 LinearRegression

In [110]:
import torch

class LinearRegressionTorch(torch.nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.linear = torch.nn.Linear(in_features, 1)
    
    def forward(self, x):
        return self.linear.forward(x)
    
    

In [112]:
X, y = make_regression_data(10000)
EPOCHS = 100
dataset = CustomDataset(X, y)

# Решил разделить датасет на тренировочную и тестовую выборки, так как по определению early_stop ориентируется на ошибку 
# валидационной выборки
train_size = int(0.8 * len(dataset)) # train
val_size = int(0.2 * len(dataset)) # validate
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# dataloader = DataLoader(
#     dataset,
#     batch_size = 128,
#     shuffle = True,
# )

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

L1_coefficient = 0.01 # Добавили коэфициент влияния штрафа на Лассо
L2_coefficient = 0.01 # Добавили коэфициент влияния штрафа на Ridge 
Lasso = 0
Ridge = 0

early_stop = EPOCHS*0.4 # этот параметр нужен, чтобы иметь лимит при обучении, в случае увеличения эпох, качество ошибки не меняется или ниже
no_improvement = 0
best_val_loss = 1e-9
delta_number = 0.0001 # Добавил, чтобы модель учитывала не все улучшения, а только значимые. А то 1^(-10) улучшение как то несерьезно


lr = 0.5
epochs = 100
model = LinearRegressionTorch(1)

loss_fn = torch.nn.MSELoss() 

optimizer = torch.optim.SGD(model.parameters(),lr=lr)

for epoch in range(1, EPOCHS+1):
    total_loss = 0
    
    for i, (batch_x, batch_y) in enumerate(train_loader):
        optimizer.zero_grad()
        y_pred = model.forward(batch_x)
        
        
        if L1_coefficient != 0 or L2_coefficient != 0:    
            Lasso = 0
            Ridge = 0
            for name, param in model.named_parameters():
                if 'bias' not in name: # в формуле Лассо и Ridge мы суммируем только веса
                    # Ошибка по формуле sum(|wi|) * coef, а после param.abs ставим .sum, так как param - тензор
                    Lasso += param.abs().sum() 
                    # Ridge это то же Лассо, только веса беруться в квадрате
                    Ridge += (param ** 2).sum()
                    
        '''
        Сделал универсальный способ учета ошибки.
        Теперь это и LassoRegression в случае если будет указан только L1, и RidgeRegression если указывать только L2, 
        и ElasticNetRegression если указан и L1 и L2, и обычная линейная регрессия если ничего не указывать 
        
        Все по формуле ElasticNet = MSE + sum(|wi|) * coef_l1 + sum(wi^2) * coef_l2 
        '''
        loss = loss_fn.forward(y_pred, batch_y) + L1_coefficient * Lasso + L2_coefficient * Ridge
        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    avg_train_loss = total_loss / len(train_loader) #ошибка на обучающей выборке 

    model.eval() #переключаем модель в режим оценки 
    total_val_loss = 0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            y_pred = model(batch_x)
            loss = loss_fn(y_pred, batch_y) # Здесь штрафы не нужны, так как мы проверям качество модели в чистом виде
            total_val_loss += loss.item()
    avg_val_loss = total_val_loss / len(val_loader)

    if avg_val_loss < best_val_loss-delta_number:
        no_improvement = 0
        best_val_loss = avg_val_loss
    else:
        no_improvement += 1

    if epoch % 10 == 0:
        log_epoch(epoch, avg_val_loss)
 
    if early_stop <= no_improvement:
        print(f"Early stopping на эпохе - {epoch}")
        break
    print(model.linear.weight.data, model.linear.bias.data)

tensor([[-4.8982]]) tensor([9.9774])
tensor([[-4.9291]]) tensor([10.0438])
tensor([[-4.9449]]) tensor([10.0008])
tensor([[-4.9556]]) tensor([9.9876])
tensor([[-4.9473]]) tensor([9.9416])
tensor([[-4.9378]]) tensor([10.0318])
tensor([[-4.9459]]) tensor([9.9775])
tensor([[-4.9077]]) tensor([10.0229])
tensor([[-4.9153]]) tensor([9.9769])
Epoch: 10	 loss:0.03885782801080495
tensor([[-4.9569]]) tensor([10.0150])
tensor([[-4.9695]]) tensor([9.9823])
tensor([[-4.9543]]) tensor([10.0079])
tensor([[-4.9195]]) tensor([10.0269])
tensor([[-5.0076]]) tensor([9.9923])
tensor([[-4.9454]]) tensor([10.0006])
tensor([[-4.9591]]) tensor([10.0147])
tensor([[-4.9189]]) tensor([9.9636])
tensor([[-4.9344]]) tensor([9.9801])
tensor([[-4.9974]]) tensor([9.9804])
Epoch: 20	 loss:0.043882836354896426
tensor([[-4.9143]]) tensor([9.9965])
tensor([[-4.9712]]) tensor([9.9653])
tensor([[-4.9756]]) tensor([10.0291])
tensor([[-4.9594]]) tensor([10.0016])
tensor([[-4.9475]]) tensor([10.0185])
tensor([[-4.9409]]) tensor(

# №1.2 LogisticRegression

In [114]:
import torch

class LogisticRegressionTorch(torch.nn.Module):
    def __init__(self, in_features, n_classes=1):
        super().__init__()
        self.linear = torch.nn.Linear(in_features, n_classes)
    
    def forward(self, x):
        return self.linear.forward(x)
    

In [130]:
#Сделаю только для бинарной классификации, сложно как то для 1.5 часа решения домашки
def accuracy(y_pred, y_true): #вынес в отдельную функцию
    y_pred = (torch.sigmoid(y_pred) > 0.5).float()
    return (y_pred == y_true).float().mean().item()

def precision(y_pred, y_true):
    y_pred = (y_pred.sigmoid() > 0.5).float()#Применяем sigmoid, чтобы получить вероятности от 0 до 1
    #это случаи (tp), когда модель сказала да (y_pred == 1) и на самом деле - да (y_true == 1) 
    tp = (y_pred * y_true).sum().item()
    #это случаи (fp), когда модель сказала да (y_pred == 1), а на самом деле - нет (y_true == 0).
    fp = ((y_pred == 1) & (y_true == 0)).sum().item()
    if tp + fp == 0:
        return 0.0
    return tp / (tp + fp)

def recall(y_pred, y_true):
    y_pred = (y_pred.sigmoid() > 0.5).float()
    tp = (y_pred * y_true).sum().item()
    fn = ((y_pred == 0) & (y_true == 1)).sum().item()
    if tp + fn == 0:
        return 0.0
    return tp / (tp + fn)

def f1_score(y_pred, y_true):
    p = precision(y_pred, y_true)
    r = recall(y_pred, y_true)
    if p + r == 0:
        return 0.0
    return 2*p*r / (p + r) #изначально по формуле 2/(1/p + 1/r), но я раскрыл скобки просто

def roc_auc(y_pred, y_true):
    y_scores = y_pred.sigmoid().detach().numpy()#выход модели до sigmoid
    y_true = y_true.numpy()#реальные метки [0, 1]
    
    #cортировка по уверенности модели
    #мы делаем список пар (score, true_label) и сортируем его по убыванию уверенности модели.
    y_scores = y_scores.reshape(-1)  #делаем 1d (на случай, если пришёл 2d)
    y_true = y_true.reshape(-1)
    scores = np.column_stack((y_scores, y_true))  # связываем предсказания и правду
    scores = scores[scores[:, 0].argsort()][::-1]  # сортировка по убыванию уверенности
    
    #Общее число положительных и отрицательных примеров
    total_pos = y_true.sum()
    total_neg = len(y_true) - total_pos

    tp = fp = 0
    tpr_list = [0]
    fpr_list = [0]

    for score in scores:
        if score[1] == 1:
            tp += 1
        else:
            fp += 1
        tpr = tp / total_pos
        fpr = fp / total_neg
        tpr_list.append(tpr)
        fpr_list.append(fpr)

    #Вычисление площади под кривой (AUC)
    auc = 0
    for i in range(1, len(tpr_list)):
        #Интегрирование методом трапеций
        auc += (fpr_list[i] - fpr_list[i - 1]) * (tpr_list[i] + tpr_list[i - 1]) / 2

    return auc


def print_confusion_matrix(y_pred, y_true):
    y_pred = (torch.sigmoid(y_pred) > 0.5).float().numpy().flatten()# Преобразуем в numpy для удобства
    y_true = y_true.numpy().flatten()

    tp = np.sum((y_pred == 1) & (y_true == 1))
    tn = np.sum((y_pred == 0) & (y_true == 0))
    fp = np.sum((y_pred == 1) & (y_true == 0))
    fn = np.sum((y_pred == 0) & (y_true == 1))

    # Выводим confusion matrix
    print("Confusion Matrix:")
    print("               | Predicted Pos | Predicted Neg |")
    print("---------------|--------------|--------------|")
    print(f"Actual Pos     |     {tp:4d}    |     {fn:4d}    |")
    print(f"Actual Neg     |     {fp:4d}    |     {tn:4d}    |")
    print("\n")

    # Считаем метрики
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    # Выводим метрики
    print(f"Accuracy : {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall   : {recall:.4f}")
    print(f"F1 Score : {f1:.4f}")

In [136]:
X, y = make_classification_data()
EPOCHS = 100
dataset = CustomDataset(X, y)
dataloader = DataLoader(
    dataset,
    batch_size = 16,
    shuffle = True,
)
lr = 0.1
epochs = 100

n_classes = len(torch.unique(y))# количество разнообразных классов
print(n_classes,'классы')

if n_classes > 2:
    model = LogisticRegressionTorch(X.shape[1], n_classes)# выбираем модель с несколькими классами
    loss_fn = torch.nn.CrossEntropyLoss() # Взял функцию для высчитывания ошибки в случае, если берем многоклассовый датасет
    '''
    С многоклассовым датасетом лучше работает Adam оптимизатор, 
    чем с SGD (SGD тоже работает нормально, но может застрять в локальных минимумах)
    '''
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
else:
    model = LogisticRegressionTorch(X.shape[1])# выбираем модель с несколькими классами
    loss_fn = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.SGD(model.parameters(),lr=lr)

# Разделение на train/val
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
    
for epoch in range(1, EPOCHS+1):
    model.train()#включаем режим обучения
    total_loss = 0
    total_acc = 0
    total_prec = 0
    total_rec = 0
    total_f1 = 0
    total_auc = 0
    for i, (batch_x, batch_y) in enumerate(train_loader):
        optimizer.zero_grad()
        y_pred = model.forward(batch_x)
        loss = loss_fn.forward(y_pred, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if n_classes > 2:
            preds = y_pred.argmax(dim=1)#Для accuracy сравниваем argmax предсказаний и реальных меток
            total_acc += (preds == batch_y).float().mean().item()
        else:
            probs = torch.sigmoid(y_pred)
            preds = (probs > 0.5).float()
            total_acc += (preds == batch_y).float().mean().item()
            
    avg_loss = total_loss / len(dataloader)
    avg_acc = total_acc/ len(dataloader)
    
    model.eval()#включаем для обучения
    all_preds = []
    all_probs = []
    all_true = []
    
    
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            y_pred_logits = model(batch_x)
            probs = torch.sigmoid(y_pred_logits)
            preds = (probs > 0.5).float().flatten()

            all_preds.extend(preds)
            all_probs.extend(probs)
            all_true.extend(batch_y)

    all_preds = torch.tensor(all_preds)
    all_probs = torch.tensor(all_probs)
    all_true = torch.tensor(all_true)

    acc = accuracy(all_probs, all_true)
    prec = precision(all_probs, all_true)
    rec = recall(all_probs, all_true)
    f1 = f1_score(all_probs, all_true)
    auc = roc_auc(all_probs, all_true)
    
    
    if epoch % 10 == 0:
        print_confusion_matrix(torch.tensor(all_probs), torch.tensor(all_true))
        log_epoch(epoch, avg_loss, accuracy=avg_acc)

print(f"Epoch {epoch} | Loss: {avg_train_loss:.4f} | "
              f"Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

2 классы


C:\Users\user\AppData\Local\Temp\ipykernel_12296\3883148379.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print_confusion_matrix(torch.tensor(all_probs), torch.tensor(all_true))


Confusion Matrix:
               | Predicted Pos | Predicted Neg |
---------------|--------------|--------------|
Actual Pos     |       72    |        0    |
Actual Neg     |       39    |        3    |


Accuracy : 0.6579
Precision: 0.6486
Recall   : 1.0000
F1 Score : 0.7869
Epoch: 10	 loss:1983.7006427447002	accuracy: 0.6260
Confusion Matrix:
               | Predicted Pos | Predicted Neg |
---------------|--------------|--------------|
Actual Pos     |       48    |       24    |
Actual Neg     |        2    |       40    |


Accuracy : 0.7719
Precision: 0.9600
Recall   : 0.6667
F1 Score : 0.7869
Epoch: 20	 loss:1264.7099523544312	accuracy: 0.6632
Confusion Matrix:
               | Predicted Pos | Predicted Neg |
---------------|--------------|--------------|
Actual Pos     |       50    |       22    |
Actual Neg     |        1    |       41    |


Accuracy : 0.7982
Precision: 0.9804
Recall   : 0.6944
F1 Score : 0.8130
Epoch: 30	 loss:1749.1208685768975	accuracy: 0.6505
Confusion 